In [ ]:
import os
import time
import osmnx as ox
import pandas as pd
from tqdm.auto import tqdm


import warnings
warnings.filterwarnings("ignore")


geonameid         : integer id of record in geonames database

name              : name of geographical point (utf8) varchar(200)

asciiname         : name of geographical point in plain ascii characters, varchar(200)

alternatenames    : alternatenames, comma separated, ascii names automatically transliterated, convenience attribute from alternatename table, varchar(10000)

latitude          : latitude in decimal degrees (wgs84)

longitude         : longitude in decimal degrees (wgs84)

feature class     : see http://www.geonames.org/export/codes.html, char(1)

feature code      : see http://www.geonames.org/export/codes.html, varchar(10)

country code      : ISO-3166 2-letter country code, 2 characters

cc2               : alternate country codes, comma separated, ISO-3166 2-letter country code, 200 characters

admin1 code       : fipscode (subject to change to iso code), see exceptions below, see file admin1Codes.txt for display names of this code; varchar(20)

admin2 code       : code for the second administrative division, a county in the US, see file admin2Codes.txt; varchar(80) 

admin3 code       : code for third level administrative division, varchar(20)

admin4 code       : code for fourth level administrative division, varchar(20)

population        : bigint (8 byte int) 

elevation         : in meters, integer

dem               : digital elevation model, srtm3 or gtopo30, average elevation of 3''x3'' (ca 90mx90m) or 30''x30'' (ca 900mx900m) area in meters, integer. srtm processed by cgiar/ciat.

timezone          : the iana timezone id (see file timeZone.txt) varchar(40)

modification date : date of last modification in yyyy-MM-dd format

In [ ]:
df = pd.read_csv('data/RU.txt', sep='\t', header=None)[[1, 14, 18]]
df.columns = ['name', 'population', 'date']
names = df[(df['population'] > 100_000) & (df['population'] < 500_000)].sort_values(by='population', ascending=True).name.values

In [ ]:
len(names)

In [ ]:
# ----------------------------
# 3. Настройки сохранения
# ----------------------------
output_folder = "graphs/Russia/100k-500k/"
os.makedirs(output_folder, exist_ok=True)

# ----------------------------
# 4. Загрузка и сохранение графов
# ----------------------------
def download_and_save_graph(city_name, folder):
    try:
        G = ox.graph_from_place(city_name, network_type="drive")
        filepath = os.path.join(folder, city_name)
        ox.save_graphml(G, filepath=filepath)
        return True  # Успешная загрузка
    except Exception as e:
        return False  # Ошибка загрузки

# ----------------------------
# 5. Цикл загрузки всех графов
# ----------------------------
with tqdm(names, desc="Загрузка городов") as pbar:
    for city in pbar:
        success = download_and_save_graph(city, output_folder)
        status = "загружен" if success else "пропущен"
        pbar.set_postfix_str(f"{city}: {status}", refresh=True)
        time.sleep(1)  # Чтобы не перегружать сервер



In [ ]:
# Путь к папке с файлами
folder_path = 'graphs/Russia/100k-500k/'

# Получаем список файлов в папке
files = os.listdir(folder_path)

# Создаем словарь для соответствия имени города и населения
name_to_population = dict(zip(df['name'], df['population']))

# Переименовываем файлы
for filename in files:
    # Убираем расширение файла (если нужно)
    
    # Проверяем, есть ли город в DataFrame
    if filename in name_to_population:
        population = name_to_population[filename]
        
        # Формируем новое имя файла
        new_filename = f"{filename}_{population}{os.path.splitext(filename)[1]}"
        
        # Полные пути к файлам
        old_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path, new_filename)
        
        # Переименовываем файл
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} -> {new_filename}")
    else:
        print(f"No population data for: {filename}, skipping")

In [11]:
import os
import osmnx as ox
from pathlib import Path

# Путь к папке с графами
graph_dir = Path('graphs/Russia/5k-15k/')

def get_place_type(place_name):
    try:
        gdf = ox.geocode_to_gdf(place_name)
        return gdf.loc[0, "addresstype"]
    except Exception as e:
        print(f"Ошибка при получении типа для {place_name}: {e}")
        return None

# Обработка файлов без расширения
for file in graph_dir.iterdir():
    if file.is_file() and file.suffix == "":
        try:
            name_part = file.name.split("_")[0]  # Извлекаем название до подчёркивания
            place_type = get_place_type(name_part)
            if place_type is None:
                continue

            new_name = f"{file.name}_{place_type}"
            new_path = file.with_name(new_name)
            os.rename(file, new_path)
            print(f"Переименовано: {file.name} → {new_name}")
        except Exception as e:
            print(f"Ошибка при обработке {file.name}: {e}")


Переименовано: Achit_5036 → Achit_5036_town
Переименовано: Agalatovo_5028 → Agalatovo_5028_town
Переименовано: Ali-Berdukovskiy_5093 → Ali-Berdukovskiy_5093_village
Переименовано: Alpatovo → Alpatovo_village
Переименовано: Amurzet → Amurzet_town
Переименовано: Andi → Andi_village
Переименовано: Andreyevo → Andreyevo_town
Переименовано: Antipino → Antipino_hamlet
Переименовано: Avsyunino → Avsyunino_town
Переименовано: Azovo → Azovo_town
Переименовано: Bada → Bada_village
Переименовано: Bamut → Bamut_village
Переименовано: Bayevo → Bayevo_town
Переименовано: Belyayevka → Belyayevka_village
Переименовано: Beya → Beya_town
Переименовано: Blagodatnoye → Blagodatnoye_village
Переименовано: Bokovskaya_5032 → Bokovskaya_5032_village
Переименовано: Borogontsy → Borogontsy_town
Переименовано: Chernyshevskiy → Chernyshevskiy_village
Переименовано: Chik_5094 → Chik_5094_town
Переименовано: Chiri-Yurt → Chiri-Yurt_village
Переименовано: Dmitrovsk → Dmitrovsk_town
Переименовано: Dorgeli → Dorgeli_m